In [16]:
import time
import copy

from pathlib import Path

import tqdm
import pandas as pd

In [17]:
from utils.load_data import load_metro_station_list


# load metro station list
path = Path.cwd().parent / 'Shanghai/metro_network_shanghai.xlsx'
data = load_metro_station_list(path, time_status='current')


# load metro station list
path = Path.cwd().parent / 'Suzhou/metro_network_suzhou.xlsx'
data = load_metro_station_list(path, time_status='current')
data

Loading data: 22it [00:02, 10.30it/s]
Loading data: 8it [00:00, 17.36it/s]


,no,name,name_zh,cumulative_distance,station_spacing,opening_date,line_name,line_name_zh,line_color,line_cycle
0,1,Mudu,木渎,0.00,0.00,2012-04-28,NaN,1号线,#78BE20,False
1,2,Jinfeng Lu,金枫路,1.14,1.14,2012-04-28,NaN,1号线,#78BE20,False
2,3,Fenhu Lu,汾湖路,2.34,1.20,2012-04-28,NaN,1号线,#78BE20,False
3,4,Yushan Lu,玉山路,3.44,1.10,2012-04-28,NaN,1号线,#78BE20,False
4,5,Shizishan,狮子山,4.68,1.24,2012-04-28,NaN,1号线,#78BE20,False
...,...,...,...,...,...,...,...,...,...,...
222,24,Lujia,菉葭,NaN,NaN,2023-06-24,NaN,11号线,#F1C6A6,False
223,25,Huaqiao Bolanzhongxin,花桥博览中心,NaN,NaN,2023-06-24,NaN,11号线,#F1C6A6,False
224,26,Jishan,集善,NaN,NaN,2023-06-24,NaN,11号线,#F1C6A6,False
225,27,Huaxigongyuan,花溪公园,NaN,NaN,2023-06-24,NaN,11号线,#F1C6A6,False


In [14]:
from utils.request_data import geocoding

key = 'd53f9929db781a22f5710539218ce798'
city = '江苏省苏州市'

all_res = []

for ix, row in tqdm.tqdm(data.iterrows()):

    ln = row['line_name_zh']
    sn = row['name_zh']
    
    search_value = f'{city}{sn}(地铁站)'

    # request result
    time.sleep(0.05)
    req_res = geocoding(search_value, city=city, key=key)

    for res in req_res:
        if res['level'] == '公交地铁站点':
            final_res = res
    else:
        final_res = req_res[0]

    # additional information
    final_res.update({
        'line_name_zh' : ln,
        'name_zh'      : sn,
        'name'         : row['name'],
        'line_name'    : row['line_name']})

    all_res.append(copy.deepcopy(final_res))

res = pd.DataFrame(all_res) 

227it [03:38,  1.04it/s]


In [15]:
res.to_csv('station_location_gcj02_suzhou.csv')